In [1]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np
import lightgbm as lgb

In [2]:
train_data = pd.read_csv('./train_noclean.csv')
test_data = pd.read_csv('./test_noclean.csv')

y_train = train_data.loc[:,'price']
train_data = train_data.drop(columns='price')
num_features = len(train_data.columns)
features = list(train_data.columns[2:num_features])
X_train = train_data.loc[:,features]

features = list(test_data.columns[2:num_features])

X_test = test_data.loc[:,features]
test_labels = test_data.loc[:,'id']

test_labels = test_labels.to_frame()

In [3]:
def run_model(X_train, y_train, learn, max_depth, num_class, num_leaves, min_data_in_leaf, rounds):
    params={}
    params['learning_rate']=0.03
    params['boosting_type']='gbdt' #GradientBoostingDecisionTree
    params['objective']='multiclass' #Multi-class target feature
    params['metric']='multi_error' #metric for multi-class
    params['max_depth']=max_depth
    params['num_class']=num_class
    params['num_leaves']=num_leaves
    clf=lgb.train(params,d_train,rounds)

In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

kf = KFold(n_splits = 10, shuffle=True)


X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)
param_grid = {
    'n_estimators': [100, 400, 700, 1000, 1500],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [15,20,25],
    'num_leaves': [10, 20, 30, 40, 50],
    'reg_lambda': [1.1, 1.2, 1.3],
    'min_split_gain': [0.3, 0.4],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20], 
    'min_data_in_leaf': [15, 20, 25, 30]
}
lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='multiclass', learning_rate=0.01, metric='multi_error')
rsearch = RandomizedSearchCV(estimator=lgb_estimator, param_distributions=param_grid, cv=kf, n_iter = 100, n_jobs = -1, verbose=10)
lgb_model = rsearch.fit(X=X, y=y)


print(lgb_model.best_params_, lgb_model.best_score_)


In [7]:
from sklearn.metrics import accuracy_score
import lightgbm as lgb
kf = KFold(n_splits = 10, shuffle=True)


X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)

for train, test in kf.split(X_train):
    X_train_kf, X_test_kf = X[train],X[test]
    y_train_kf, y_test_kf = y[train],y[test]
    d_train=lgb.Dataset(X_train_kf, label=y_train_kf)
    best_param = {'verbose':-1,'boosting_type':'gbdt',  'objective':'multiclass', 'learning_rate':0.01, 'num_class':5, 'metric':'multi_error','subsample_freq': 20, 'subsample': 0.8, 'reg_lambda': 1.2, 'num_leaves': 40, 'n_estimators': 1000, 'min_split_gain': 0.4, 'min_data_in_leaf': 25, 'max_depth': 20, 'colsample_bytree': 0.7}
    clf=lgb.train(best_param,d_train )
    
    y_pred=clf.predict(X_test_kf)
    y_pred = [np.argmax(line) for line in y_pred]
    print(accuracy_score(y_test_kf, y_pred))

0.5490196078431373
0.5413223140495868
0.5712809917355371
0.5609504132231405
0.5423553719008265
0.5599173553719008
0.5547520661157025
0.5568181818181818
0.5599173553719008
0.5712809917355371


In [8]:
X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)

best_param = {'boosting_type':'gbdt',  'objective':'multiclass', 'learning_rate':0.01, 'num_class':5, 'metric':'multi_error','subsample_freq': 20, 'subsample': 0.8, 'reg_lambda': 1.2, 'num_leaves': 40, 'n_estimators': 1000, 'min_split_gain': 0.4, 'min_data_in_leaf': 25, 'max_depth': 20, 'colsample_bytree': 0.7}
d_train=lgb.Dataset(X, y)
clf=lgb.train(best_param,d_train)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002107 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1519
[LightGBM] [Info] Number of data points in the train set: 9681, number of used features: 50
[LightGBM] [Info] Start training from score -34.538776
[LightGBM] [Info] Start training from score -1.385985
[LightGBM] [Info] Start training from score -1.388052
[LightGBM] [Info] Start training from score -1.367568
[LightGBM] [Info] Start training from score -1.403905
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

In [9]:
y_pred1=clf.predict(X)
y_pred1 = [np.argmax(line) for line in y_pred1]
print(accuracy_score(y, y_pred1))

0.8393760975105877


In [10]:
y_pred=clf.predict(X_test)

In [11]:
print(y_pred)

[[9.63501347e-16 1.65708195e-01 5.21646525e-01 2.54137479e-01
  5.85078004e-02]
 [9.88058645e-16 6.23022411e-02 3.79039465e-01 4.62610231e-01
  9.60480631e-02]
 [9.40101224e-16 2.29830443e-01 4.57659098e-01 2.62552010e-01
  4.99584497e-02]
 ...
 [9.69967917e-16 7.50665617e-02 2.24979721e-01 4.05296374e-01
  2.94657343e-01]
 [3.57829217e-16 9.08182000e-01 4.16185959e-02 3.26919409e-02
  1.75074634e-02]
 [1.39995573e-15 1.31422473e-01 3.79822158e-01 3.36081396e-01
  1.52673973e-01]]


In [12]:
y_pred = [np.argmax(line) for line in y_pred]

In [13]:
print(y_pred)

[2, 3, 2, 2, 2, 4, 4, 1, 3, 2, 1, 1, 1, 3, 1, 1, 1, 2, 2, 4, 3, 2, 3, 4, 2, 4, 4, 4, 1, 4, 3, 3, 2, 1, 3, 4, 4, 1, 1, 1, 4, 4, 4, 1, 4, 4, 4, 1, 2, 2, 2, 3, 1, 1, 1, 3, 1, 4, 2, 3, 2, 4, 1, 3, 1, 4, 1, 4, 2, 3, 4, 3, 2, 3, 3, 2, 4, 3, 1, 1, 2, 3, 1, 3, 2, 4, 1, 3, 1, 1, 2, 4, 3, 2, 1, 4, 4, 2, 3, 2, 1, 1, 2, 4, 1, 1, 1, 3, 1, 1, 3, 1, 4, 3, 3, 3, 4, 4, 2, 1, 1, 4, 2, 1, 2, 3, 3, 1, 1, 4, 3, 3, 3, 3, 1, 4, 3, 4, 4, 2, 4, 3, 4, 4, 4, 3, 4, 4, 2, 4, 1, 1, 4, 2, 3, 2, 3, 3, 4, 1, 3, 2, 3, 4, 3, 3, 2, 4, 1, 1, 4, 1, 1, 4, 1, 2, 2, 4, 4, 3, 2, 4, 4, 2, 2, 3, 1, 3, 2, 3, 1, 2, 1, 3, 1, 1, 3, 1, 1, 1, 4, 3, 1, 1, 2, 3, 2, 3, 4, 3, 2, 1, 2, 2, 4, 3, 2, 3, 3, 4, 2, 4, 2, 1, 1, 3, 3, 4, 2, 4, 3, 3, 3, 3, 3, 4, 3, 2, 4, 1, 2, 1, 2, 2, 4, 2, 1, 2, 4, 4, 2, 1, 1, 1, 4, 1, 2, 2, 2, 3, 3, 2, 3, 1, 2, 1, 1, 1, 2, 2, 2, 4, 1, 1, 3, 3, 1, 1, 3, 3, 2, 2, 3, 4, 2, 2, 3, 3, 2, 4, 2, 3, 1, 2, 3, 1, 3, 2, 1, 4, 4, 3, 4, 1, 2, 3, 1, 1, 3, 3, 4, 2, 4, 1, 3, 4, 4, 1, 3, 4, 1, 1, 3, 2, 2, 3, 1, 1, 3, 2, 3, 3, 3, 

In [14]:
df = pd.DataFrame
test_labels['price'] = y_pred

test_labels['price'] = test_labels['price'].map(lambda x: int(x))

In [15]:
test_labels

,id,price
0,7715,2
1,13196,3
2,13194,2
3,4673,2
4,11325,2
...,...,...
4144,12921,2
4145,7174,1
4146,9240,3
4147,11663,1


In [16]:
test_labels.to_csv(r'./submission7.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV

kf = KFold(n_splits = 10, shuffle=True)


X = np.concatenate([X_train], axis= 0)
y = np.concatenate([y_train], axis= 0)
param_grid = {
    'n_estimators': [100, 400, 700, 1000],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [8,10,20],
    'num_leaves': [15, 100, 200],
    'reg_alpha': [1.1, 1.2, 1.3],
    'reg_lambda': [1.1, 1.2, 1.3],
    'min_split_gain': [0.3, 0.4],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20]
}
lgb_estimator = lgb.LGBMClassifier(boosting_type='gbdt',  objective='multiclass', learning_rate=0.01, metric='multi_error')
gsearch = GridSearchCV(estimator=lgb_estimator, param_grid=param_grid, cv=kf, verbose=2, n_jobs=-1)
lgb_model = gsearch.fit(X=X, y=y)


print(lgb_model.best_params_, lgb_model.best_score_)
